<a href="https://colab.research.google.com/github/DiploDatos/AprendizajeProfundo/blob/master/4_mlflow_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLflow y un experimento sencillo

## Registro de experimentos con MLflow

A la hora de trabajar con problemas de aprendizaje supervisado en general, y de aprendizaje profundo en particular, es necesario llevar un buen registro de los experimentos realizados. En particular, cuando se hace uso de algún tipo de optimización de hiperparámetros (ya sea búsqueda exhaustiva, búsqueda aleatoria o algo más complejo como optimización bayesiana), es crucial tener registro de que se fue haciendo para tomar las decisiones.

[MLflow](https://mlflow.org/) es una plataforma de código abierto que facilita mucho lleva y cuenta de los experimentos que se están realizando. Se pueden ejecutar experimentos y guardar hiperparámetros y métricas de los mismos.

A continuación vamos a ver un ejemplo muy sencillo de como realizar un experimento utilizando MLflow y el conjunto de datos de IMDB.

### Descarga de datos 

#### Para quienes trabajen en Google colab

Si están trabajando en Google Colab, tienen 2 opciones para trabajar con los datos de esta notebook.

1. Pueden descargar los datos necesiten descargar los datos en su computadora usando los siguientes links
- https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/glove.6B.50d.txt.gz
- https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/imdb_reviews.csv.gz

y luego subirlos a una carpeta de drive. Una vez que los tengan en drive puede leerlos usando los siguientes fragmentos de código:



```
#Esto monta todo lo que tienen en Google Drive en Google Colab temporalmente (es decir, por el tiempo que dure la sesion)
from google.colab import drive 
drive.mount('/content/drive')
```

Ahora obtenemos la url de su directorio de drive para cada dataset. Para esto, tienen que navegar por el panel izquierdo de la notebook de colab, ubicar el archivo y copiar la ruta del mismo (ver [esta imagen](https://media-exp1.licdn.com/dms/image/C4E12AQHGB6c6qitRrQ/article-inline_image-shrink_1500_2232/0/1643815540061?e=1668643200&v=beta&t=W32ew3DXU_GKNc0J_Gr4mQfmQ0X7NTs-809UL5V8hsg) para más detalle).



La url debería quedarles algo similar a lo siguiente (puede variar dependiendo la cantidad de carpetas que tengan hasta llegar al archivo):



```
url_glove = '/content/drive/MyDrive/DIPLO_DATOS/glove.6B.50d.txt.gz'
url_imdb_reviews = '/content/drive/MyDrive/DIPLO_DATOS/imdb_reviews.csv.gz'
```







2. La opción 2 evita todos los pasos anteriores pero tiene la desventaja de que descarga los datos cada vez que queremos usar la notebook (si el o los datasets son grandes esto puede hacernos perder tiempo). Dado que estamos trabajando con datasets relativamente pequeños, en esta notebook vamos a utilizar esta opción. Por lo tanto leemos los datos directamente sin ningún path.

#### Para quienes estén trabajando en nabu

Si ya usaron nabu en las notebooks anteriores seguro hayan descargando los datos en la carpeta `data` mediante comando de terminal usando las siguientes lineas:

```
curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/glove.6B.50d.txt.gz -o ./data/glove.6B.50d.txt.gz
curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/imdb_reviews.csv.gz -o ./data/imdb_reviews.csv.gz
```


En tal caso, su dirección de los datos será algo parecido a lo siguiente:




```
url_glove = "./data/glove.6B.50d.txt.gz"
url_imdb_reviews = "./data/imdb_reviews.csv.gz"
```







In [1]:
!curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/glove.6B.50d.txt.gz -o glove.6B.50d.txt.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 65.9M  100 65.9M    0     0  12.3M      0  0:00:05  0:00:05 --:--:-- 13.2M


In [2]:
!curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/imdb_reviews.csv.gz -o imdb_reviews.csv.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.3M  100 25.3M    0     0  10.5M      0  0:00:02  0:00:02 --:--:-- 10.5M


In [3]:
## Esta linea sirve principalmente para quienes estén trabajando en colab ya que colab tiene problemas de actualización con gensim
!pip install gensim --upgrade 


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [4]:
# Idem al caso anterior, en caso que no tengan instalado mflow o estén corriendo en google colab ejecutar la siguiente linea
!pip install mlflow 


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


### Librerías

In [5]:
import gzip
import mlflow
import pandas as pd
import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from gensim import corpora
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm, trange

## Dataset

Nos basaremos en el dataset que creamos para el [notebook 2](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/3_datasets.ipynb), con la diferencia de que pasaremos el dataframe de manera directa (esto es para poder hacer split en train/test).

Recordemos de la [notebook 2](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/3_datasets.ipynb) que la clase abstracta `torch.utils.data.Dataset `es la [clase base](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html?highlight=dataset) para construir un dataset de `PyTorch`. Cualquier dataset personalizado debe heredar de dicha clase e implementar los siguientes métodos:

`__len__`: Para que el comando `len(dataset)` devuelva el tamaño del conjunto de datos.

`__getitem__`: Para soportar indexado de manera que `dataset[i]` devuelva el elemento `i`.

In [6]:
class IMDBReviewsDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.to_list()
        
        item = {
            "data": self.dataset.loc[item, "review"],
            "target": self.dataset.loc[item, "sentiment"]
        }
        
        if self.transform:
            item = self.transform(item)
        
        return item

## Preprocesamiento

En este caso vamos a utilizar un sólo módulo para transformar los datos de IMDB. Este se encargará de preprocesar el texto (i.e. normalizarlo) y transformará las palabras en índices de un diccionario para luego poder pasar una secuencia de palabras para buscar en la matriz de embeddings y así permitir mayor manipulación de los embeddings (en lugar de utilizar embeddings fijos).

Vamos a estar trabajando con la librería [gensim](https://pypi.org/project/gensim/) previamente importada para el procesamiento del lenguaje natural (pueden ver su código open source en este [link](https://github.com/RaRe-Technologies/gensim) ).



In [7]:
class RawDataProcessor:
    def __init__(self, 
                 dataset, 
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
        # Create dictionary based on all the reviews (with corresponding preprocessing)
        # https://radimrehurek.com/gensim/corpora/dictionary.html
        self.dictionary = corpora.Dictionary(
            dataset["review"].map(self._preprocess_string).tolist()
        )
        # Filter the dictionary with extremos words
        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.filter_extremes.html?highlight=filter_extrem
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        
        # Make the indices continuous after some words have been removed
        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.compactify.html
        self.dictionary.compactify()
        
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        })
        self.idx_to_target = sorted(dataset["sentiment"].unique())
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}


    def _preprocess_string(self, string):
        # https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.preprocess_string:~:text=gensim.parsing.preprocessing.preprocess_string
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
      # https://radimrehurek.com/gensim/corpora/dictionary.html#:~:text=doc2idx(document,via%20unknown_word_index.
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx[target]
    
    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self.encode_data(item["data"])
        else:
            data = [self.encode_data(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self.encode_target(item["target"])
        else:
            target = [self.encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

## Lectura de datos

En esta ocasión, leeremos los datos de IMDB y lo dividiremos en subconjuntos de entrenamiento y evaluación.

In [8]:
dataset = pd.read_csv("imdb_reviews.csv.gz")

preprocess = RawDataProcessor(dataset)

train_indices, test_indices = train_test_split(dataset.index, test_size=0.2, random_state=42)

train_dataset = IMDBReviewsDataset(dataset.loc[train_indices].reset_index(drop=True), transform=preprocess)

test_dataset = IMDBReviewsDataset(dataset.loc[test_indices].reset_index(drop=True), transform=preprocess)

print(f"Datasets loaded with {len(train_dataset)} training elements and {len(test_dataset)} test elements")
print(f"Sample train element:\n{train_dataset[0]}")

Datasets loaded with 40000 training elements and 10000 test elements
Sample train element:
{'data': [1068, 2734, 1667, 5702, 16275, 9487, 1531, 7917, 32915, 653, 4760, 1303, 881, 2363, 189, 146, 1108, 9235, 1811, 557, 1540, 4915, 182, 34361, 303, 6125, 42193, 14521, 2170, 530, 7746, 861, 9056, 25504, 449, 479, 258, 3929, 1, 30520, 2011, 14186, 290, 1800, 1, 1, 69, 3007, 531, 1030, 1024, 314, 283, 760, 20011, 7858, 7205, 2777, 13861, 1, 1, 1915, 1139, 1409, 6580, 295, 9, 2893, 1922, 387, 871, 2012, 174, 7637, 301, 3694, 3758, 233, 20587, 8971, 516, 4806, 7077, 880, 9543, 5478, 4153, 32033, 488, 1037, 624, 2622, 5953, 10022, 11171, 5994, 265, 1317, 29065, 1085, 5379, 2296, 19816, 11980, 912, 784, 5040, 24689, 477, 6325, 11427, 16648, 43279, 3804, 1, 22381, 545, 1141, 6404, 1413, 8580, 9089, 303, 4922, 2598, 3271, 820, 1198, 1152, 8810, 196, 2130, 8119, 4506, 19833, 721, 306, 27605, 291, 2445, 2984, 6777, 38709, 17240, 909, 39, 1439, 11505, 201, 1900, 11736, 600, 215, 1807, 22, 6646, 1895

## Collation function

Como en este caso trabajamos con secuencias de palabras (representadas por sus índices en un vocabulario), cuando queremos buscar un *batch* de datos, el `DataLoader` de PyTorch espera que los datos del *batch* tengan la misma dimensión (para poder llevarlos todos a un tensor de dimensión fija). Esto lo podemos lograr mediante el parámetro de `collate_fn`. En particular, esta función se encarga de tomar varios elementos de un `Dataset` y combinarlos de manera que puedan ser devueltos como un tensor de PyTorch. Muchas veces la `collate_fn` que viene por defecto en `DataLoader` sirve (como se vio en el notebook 2), pero este no es el caso. Se define un módulo `PadSequences` que toma un valor mínimo, opcionalmente un valor máximo y un valor de relleno (*pad*) y dada una lista de secuencias, devuelve un tensor con *padding* sobre dichas secuencias.

In [9]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.FloatTensor(target)
        }

## DataLoaders

Ya habiendo definido nuestros conjuntos de datos y nuestra `collation_fn`, podemos definir nuestros `DataLoader`, uno para entrenamiento y otro para evaluación. Ver que la diferencia fundamental está en `shuffle`, no queremos mezclar los valores de evaluación cada vez que evaluamos porque al evaluar mediante *mini-batchs* nos puede generar inconsistencias.

In [10]:
pad_sequences = PadSequences()
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True,
                          collate_fn=pad_sequences, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False,
                         collate_fn=pad_sequences, drop_last=False)

## El modelo de clasificación

Para clasificación utilizaremos un perceptrón multicapa de dos capas ocultas. Claramente este modelo es naive y prácticamente todo lo que está *hardcodeado* (e.g. los tamaños de las capas o la cantidad de capas) podría ser parte de los parámetros del modelo. En particular, tenemos la capa de `Embeddings` que es rellenada con los valores de embeddings preentrenados (los de Glove en este caso).

In [11]:
class IMDBReviewsClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with gzip.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden1 = nn.Linear(vector_size, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.output = nn.Linear(128, 1)
        self.vector_size = vector_size
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = torch.sigmoid(self.output(x))
        return x

## Experimento de MLflow

Por último, ya tenemos todos los bloques para construir nuestro experimento de MLflow. Anotamos un par de parámetros  (estos pueden ser todos los que se consideren necesarios) y lanzamos a correr nuestro experimento. Cada vez que finaliza un epoch guardamos algunas métricas. Al finalizar todos los epochs corremos algunas métricas extras de evaluación y guardamos algunos datos extra que nos servirán para calcular otras métricas a futuro.

Material auxiliar:

-  [notebook](https://github.com/PiConsulting/Pensadero/blob/master/Eventos/2020/2020-03-06_wids_mlflow/mlflow_basic.ipynb) con un ejemplo clásico usando sklearn.


In [12]:
mlflow.set_experiment("a_naive_experiment")

with mlflow.start_run():
    mlflow.log_param("model_name", "mlp")
    mlflow.log_param("freeze_embedding", True)
    mlflow.log_params({
        "embedding_size": 50,
        "hidden1_size": 128,
        "hidden2_size": 128
    })
    model = IMDBReviewsClassifier("glove.6B.50d.txt.gz", preprocess.dictionary, 50, True)
    loss = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    for epoch in trange(3):
        model.train()
        running_loss = []
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            output = model(batch["data"])
            loss_value = loss(output, batch["target"].view(-1, 1))
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())        
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
        
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            output = model(batch["data"])
            running_loss.append(
                loss(output, batch["target"].view(-1, 1)).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.squeeze().detach().numpy())
        mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric("test_avp", average_precision_score(targets, predictions), epoch)
    
    with tempfile.TemporaryDirectory() as tmpdirname:
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            output = model(batch["data"])
            targets.extend(batch["target"].numpy())
            predictions.extend(output.squeeze().detach().numpy())
        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
            f"{tmpdirname}/predictions.csv.gz", index=False
        )
        mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]